In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
#from datetime import datetime
import matplotlib.pyplot as plt
sns.set(style="darkgrid")
plt.style.use('seaborn-notebook')
from scipy import stats
import math
import os
from dateutil.parser import parse
from datetime import datetime, date, time
import calendar
from datetime import timedelta

In [2]:
#Data loading
dataset2 = pd.read_excel(open('Online Retail.xlsx','rb'), sheet_name='Online Retail')


In [3]:
#Downsampling if needed
dataset = dataset2#[:5000]
dataset.shape

(541909, 8)

In [4]:
#Create a sub-sample
dataset.to_csv('ordersample.csv')

In [5]:
#We sort the dataset and check the time range
dataset = dataset.sort_values('InvoiceDate')
print('First order', dataset['InvoiceDate'].min())
print('Last order', dataset['InvoiceDate'].max())

First order 2010-12-01 08:26:00
Last order 2011-12-09 12:50:00


In [6]:
#Day 1 definition
today = datetime(2012,1,1,0,0)

In [7]:
#We mark the cancelled invoices
dataset['Cancelled'] = dataset['InvoiceNo'].apply(lambda x: 1 if str(x).startswith('C') else 0)
dataset[dataset['Cancelled'] == 1].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,1
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,1
241,C536391,22553,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,1
239,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548.0,United Kingdom,1
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,1


In [8]:
#We keep only the info from UK
dataset = dataset[dataset['Country'] == 'United Kingdom']
dataset.shape

(495478, 9)

In [9]:
#We drop the duplicates
dataset = dataset.drop_duplicates()
dataset.shape

(490300, 9)

In [10]:
#We drop the fields with no customer ID
dataset = dataset.dropna(axis = 0, how = 'any', subset = ['CustomerID'])
dataset.shape

(356728, 9)

In [11]:
#Any odd values ?
dataset.describe()

,Quantity,UnitPrice,CustomerID,Cancelled
count,356728.000000,356728.000000,356728.000000,356728.000000
mean,11.198644,3.268255,15543.795284,0.021027
std,264.998044,71.162330,1594.286219,0.143475
min,-80995.000000,0.000000,12346.000000,0.000000
25%,2.000000,1.250000,14191.000000,0.000000
50%,4.000000,1.950000,15513.000000,0.000000
75%,12.000000,3.750000,16931.000000,0.000000
max,80995.000000,38970.000000,18287.000000,1.000000


In [12]:
#We clean all the weird Stockcodes

#drop all the that are extra fees
extrafeereasons = ['POST', 'DOT', 'C2', 'CRUK', 'M', 'BANK CHARGES', 'PADS', 'D']
for todrop in extrafeereasons:
    dataset = dataset[dataset['StockCode'] != todrop]
dataset.shape


(356110, 9)

In [13]:
dataset = dataset[dataset['UnitPrice'] != 0.00]
dataset.shape

(356090, 9)

In [14]:
#We clean the quantity < 0 and invoice not cancelled
#dataset = 
dataset[(dataset['Quantity'] < 0) & (dataset['Cancelled'] == 0)]
#dataset.shape

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled


In [15]:
#No free stuff in the dataset, must cost at least 1 penny
dataset = dataset[dataset['UnitPrice'] >= 0.01]
dataset.shape

(356090, 9)

In [16]:
#Any np.nan remaining ?
dataset.isnull().sum(axis=0)

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
Cancelled      0
dtype: int64

In [17]:
#Totat per item
dataset['TotalItem'] = dataset['UnitPrice'] * dataset['Quantity']
dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled,TotalItem
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34


In [18]:
#initial price calculation
initpricedict = dataset[['UnitPrice', 'StockCode']].groupby('StockCode')['UnitPrice'].first().to_dict()
dataset['Initprice'] = dataset['StockCode'].apply(lambda x : initpricedict[x])
dataset['ItemDiscount'] = dataset['Initprice'] / dataset['UnitPrice']
dataset['TotalItemInit'] = dataset['Initprice'] * dataset['Quantity']
dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled,TotalItem,Initprice,ItemDiscount,TotalItemInit
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,15.30,2.55,1.0,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,22.00,2.75,1.0,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34


In [19]:
#Savings calculation
dataset['Savings'] = dataset['TotalItem'] - dataset['TotalItemInit']
dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled,TotalItem,Initprice,ItemDiscount,TotalItemInit,Savings
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,15.30,2.55,1.0,15.30,0.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34,0.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,22.00,2.75,1.0,22.00,0.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34,0.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34,0.0


In [20]:
dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled,TotalItem,Initprice,ItemDiscount,TotalItemInit,Savings
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,15.30,2.55,1.0,15.30,0.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34,0.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,22.00,2.75,1.0,22.00,0.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34,0.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34,0.0


In [21]:
#We split the unit prices by quantile
classes = 5

#### CALULATION BEGINNING ####
divider = 100/classes

price = dataset[['StockCode', 'UnitPrice']].groupby('StockCode').first()

price['QuantUnitPrice'] = price['UnitPrice'].apply(lambda x: math.ceil(stats.percentileofscore(price['UnitPrice'],x, kind = 'mean')/divider))
price[['UnitPrice', 'QuantUnitPrice']].head()


pricedictquant = price['QuantUnitPrice'].to_dict()

dataset['QuantUnitPrice'] = dataset['StockCode'].apply(lambda x: pricedictquant.get(x, 0))

dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled,TotalItem,Initprice,ItemDiscount,TotalItemInit,Savings,QuantUnitPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,15.30,2.55,1.0,15.30,0.0,3
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34,0.0,4
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,22.00,2.75,1.0,22.00,0.0,4
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34,0.0,4
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34,0.0,4


In [22]:
#quant unit price vectorization - dollars

temp = dataset.pivot(columns='QuantUnitPrice', values='TotalItem').fillna(0)
new_names = [(i, 'QuantUnitPrice_{:02d}'.format(i)) for i in temp.columns.values]
temp.rename(columns = dict(new_names), inplace=True)
temp.head()
dataset = dataset.merge(temp, how='inner', left_index=True, right_index=True)
dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled,TotalItem,Initprice,ItemDiscount,TotalItemInit,Savings,QuantUnitPrice,QuantUnitPrice_01,QuantUnitPrice_02,QuantUnitPrice_03,QuantUnitPrice_04,QuantUnitPrice_05
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,15.30,2.55,1.0,15.30,0.0,3,0.0,0.0,15.3,0.00,0.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34,0.0,4,0.0,0.0,0.0,20.34,0.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,22.00,2.75,1.0,22.00,0.0,4,0.0,0.0,0.0,22.00,0.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34,0.0,4,0.0,0.0,0.0,20.34,0.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,3.39,1.0,20.34,0.0,4,0.0,0.0,0.0,20.34,0.0


In [23]:
#quant unit price savings vectorization - savings

temp = dataset.pivot(columns='QuantUnitPrice', values='Savings').fillna(0)
new_names = [(i, 'QuantUnitSavings_{:02d}'.format(i)) for i in temp.columns.values]
temp.rename(columns = dict(new_names), inplace=True)
temp.head()
dataset = dataset.merge(temp, how='inner', left_index=True, right_index=True)
dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled,TotalItem,...,QuantUnitPrice_01,QuantUnitPrice_02,QuantUnitPrice_03,QuantUnitPrice_04,QuantUnitPrice_05,QuantUnitSavings_01,QuantUnitSavings_02,QuantUnitSavings_03,QuantUnitSavings_04,QuantUnitSavings_05
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,15.30,...,0.0,0.0,15.3,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,...,0.0,0.0,0.0,20.34,0.0,0.0,0.0,0.0,0.0,0.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,22.00,...,0.0,0.0,0.0,22.00,0.0,0.0,0.0,0.0,0.0,0.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,...,0.0,0.0,0.0,20.34,0.0,0.0,0.0,0.0,0.0,0.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,...,0.0,0.0,0.0,20.34,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
#Amount cancelled
dataset['AmountCancelled'] = dataset['Cancelled'] * dataset['TotalItem']

dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cancelled,TotalItem,...,QuantUnitPrice_02,QuantUnitPrice_03,QuantUnitPrice_04,QuantUnitPrice_05,QuantUnitSavings_01,QuantUnitSavings_02,QuantUnitSavings_03,QuantUnitSavings_04,QuantUnitSavings_05,AmountCancelled
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,15.30,...,0.0,15.3,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,...,0.0,0.0,20.34,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,22.00,...,0.0,0.0,22.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,...,0.0,0.0,20.34,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,...,0.0,0.0,20.34,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
dataset.columns.values

array(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Cancelled', 'TotalItem',
       'Initprice', 'ItemDiscount', 'TotalItemInit', 'Savings',
       'QuantUnitPrice', 'QuantUnitPrice_01', 'QuantUnitPrice_02',
       'QuantUnitPrice_03', 'QuantUnitPrice_04', 'QuantUnitPrice_05',
       'QuantUnitSavings_01', 'QuantUnitSavings_02', 'QuantUnitSavings_03',
       'QuantUnitSavings_04', 'QuantUnitSavings_05', 'AmountCancelled'], dtype=object)

In [42]:
#Invoicedb calculation

def my_agg(x):
    aggcalcul = {
        'InvoiceDate': x['InvoiceDate'].min(),
        'TotalInvoice': x['TotalItem'].sum(),
        'TotalInvoiceInit': x['TotalItemInit'].sum(),
        'CustomerID':  x['CustomerID'].min(),
        'TotalSavings': x['Savings'].sum(),
        'AmountCancelled' : x['AmountCancelled'].sum(),
        }
    return pd.Series(aggcalcul, index=aggcalcul.keys())
invoicedb = dataset.groupby('InvoiceNo').apply(my_agg).fillna(0)

invoicedb.head()


,InvoiceDate,TotalInvoice,TotalInvoiceInit,CustomerID,TotalSavings,AmountCancelled
InvoiceNo,,,,,,
536365,2010-12-01 08:26:00,139.12,139.12,17850.0,0.0,0.0
536366,2010-12-01 08:28:00,22.20,22.20,17850.0,0.0,0.0
536367,2010-12-01 08:34:00,278.73,278.73,13047.0,0.0,0.0
536368,2010-12-01 08:34:00,70.05,70.05,13047.0,0.0,0.0
536369,2010-12-01 08:35:00,17.85,17.85,13047.0,0.0,0.0


In [43]:
#detail orders for invoicedb - QUANT UNIT PRICE
detail = [x for x in dataset.columns.values if x.startswith('QuantUnitPrice_')]
detail.append('InvoiceNo')
temp = dataset[detail].groupby('InvoiceNo').sum()
temp.head()
invoicedb = invoicedb.merge(temp, how='inner', left_index=True, right_index=True)
invoicedb.head()

,InvoiceDate,TotalInvoice,TotalInvoiceInit,CustomerID,TotalSavings,AmountCancelled,QuantUnitPrice_01,QuantUnitPrice_02,QuantUnitPrice_03,QuantUnitPrice_04,QuantUnitPrice_05
InvoiceNo,,,,,,,,,,,
536365,2010-12-01 08:26:00,139.12,139.12,17850.0,0.0,0.0,0.0,0.0,15.30,108.52,15.30
536366,2010-12-01 08:28:00,22.20,22.20,17850.0,0.0,0.0,0.0,0.0,22.20,0.00,0.00
536367,2010-12-01 08:34:00,278.73,278.73,13047.0,0.0,0.0,0.0,0.0,89.18,55.50,134.05
536368,2010-12-01 08:34:00,70.05,70.05,13047.0,0.0,0.0,0.0,0.0,0.00,25.50,44.55
536369,2010-12-01 08:35:00,17.85,17.85,13047.0,0.0,0.0,0.0,0.0,0.00,0.00,17.85


In [44]:
#detail orders for invoicedb - QUANT UNIT SAVINGS
detail = [x for x in dataset.columns.values if x.startswith('QuantUnitSavings_')]
detail.append('InvoiceNo')
temp = dataset[detail].groupby('InvoiceNo').sum()
temp.head()
invoicedb = invoicedb.merge(temp, how='inner', left_index=True, right_index=True)
invoicedb.head()

,InvoiceDate,TotalInvoice,TotalInvoiceInit,CustomerID,TotalSavings,AmountCancelled,QuantUnitPrice_01,QuantUnitPrice_02,QuantUnitPrice_03,QuantUnitPrice_04,QuantUnitPrice_05,QuantUnitSavings_01,QuantUnitSavings_02,QuantUnitSavings_03,QuantUnitSavings_04,QuantUnitSavings_05
InvoiceNo,,,,,,,,,,,,,,,,
536365,2010-12-01 08:26:00,139.12,139.12,17850.0,0.0,0.0,0.0,0.0,15.30,108.52,15.30,0.0,0.0,0.0,0.0,0.0
536366,2010-12-01 08:28:00,22.20,22.20,17850.0,0.0,0.0,0.0,0.0,22.20,0.00,0.00,0.0,0.0,0.0,0.0,0.0
536367,2010-12-01 08:34:00,278.73,278.73,13047.0,0.0,0.0,0.0,0.0,89.18,55.50,134.05,0.0,0.0,0.0,0.0,0.0
536368,2010-12-01 08:34:00,70.05,70.05,13047.0,0.0,0.0,0.0,0.0,0.00,25.50,44.55,0.0,0.0,0.0,0.0,0.0
536369,2010-12-01 08:35:00,17.85,17.85,13047.0,0.0,0.0,0.0,0.0,0.00,0.00,17.85,0.0,0.0,0.0,0.0,0.0


In [45]:
#Invoicedb updates

#Day between orders calculation
invoicedb = invoicedb.sort_values('InvoiceDate')
invoicedb['Ordersep'] = invoicedb[['CustomerID', 'InvoiceDate']].groupby(['CustomerID']).InvoiceDate.apply(lambda x: x.diff()).fillna(0)
invoicedb['Ordersep'] = invoicedb['Ordersep'].apply(lambda x: x.days)
invoicedb.head()

,InvoiceDate,TotalInvoice,TotalInvoiceInit,CustomerID,TotalSavings,AmountCancelled,QuantUnitPrice_01,QuantUnitPrice_02,QuantUnitPrice_03,QuantUnitPrice_04,QuantUnitPrice_05,QuantUnitSavings_01,QuantUnitSavings_02,QuantUnitSavings_03,QuantUnitSavings_04,QuantUnitSavings_05,Ordersep
InvoiceNo,,,,,,,,,,,,,,,,,
536365,2010-12-01 08:26:00,139.12,139.12,17850.0,0.0,0.0,0.0,0.0,15.30,108.52,15.30,0.0,0.0,0.0,0.0,0.0,0
536366,2010-12-01 08:28:00,22.20,22.20,17850.0,0.0,0.0,0.0,0.0,22.20,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0
536367,2010-12-01 08:34:00,278.73,278.73,13047.0,0.0,0.0,0.0,0.0,89.18,55.50,134.05,0.0,0.0,0.0,0.0,0.0,0
536368,2010-12-01 08:34:00,70.05,70.05,13047.0,0.0,0.0,0.0,0.0,0.00,25.50,44.55,0.0,0.0,0.0,0.0,0.0,0
536369,2010-12-01 08:35:00,17.85,17.85,13047.0,0.0,0.0,0.0,0.0,0.00,0.00,17.85,0.0,0.0,0.0,0.0,0.0,0


In [46]:
#InvoiceDB discount
invoicedb['Discount'] = invoicedb['TotalInvoice'] / invoicedb['TotalInvoiceInit']
invoicedb.head()

,InvoiceDate,TotalInvoice,TotalInvoiceInit,CustomerID,TotalSavings,AmountCancelled,QuantUnitPrice_01,QuantUnitPrice_02,QuantUnitPrice_03,QuantUnitPrice_04,QuantUnitPrice_05,QuantUnitSavings_01,QuantUnitSavings_02,QuantUnitSavings_03,QuantUnitSavings_04,QuantUnitSavings_05,Ordersep,Discount
InvoiceNo,,,,,,,,,,,,,,,,,,
536365,2010-12-01 08:26:00,139.12,139.12,17850.0,0.0,0.0,0.0,0.0,15.30,108.52,15.30,0.0,0.0,0.0,0.0,0.0,0,1.0
536366,2010-12-01 08:28:00,22.20,22.20,17850.0,0.0,0.0,0.0,0.0,22.20,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0,1.0
536367,2010-12-01 08:34:00,278.73,278.73,13047.0,0.0,0.0,0.0,0.0,89.18,55.50,134.05,0.0,0.0,0.0,0.0,0.0,0,1.0
536368,2010-12-01 08:34:00,70.05,70.05,13047.0,0.0,0.0,0.0,0.0,0.00,25.50,44.55,0.0,0.0,0.0,0.0,0.0,0,1.0
536369,2010-12-01 08:35:00,17.85,17.85,13047.0,0.0,0.0,0.0,0.0,0.00,0.00,17.85,0.0,0.0,0.0,0.0,0.0,0,1.0


In [47]:
#When the order has been placed during the day in pounds?

def daysplit(x):
    hour = x.hour
    if 6 < hour < 12:
        return 1
    elif 12 <= hour < 20:
        return 2
    else:
        return 3

invoicedb['Daytime'] = invoicedb['InvoiceDate'].apply(daysplit)
    
temp = invoicedb.pivot(columns='Daytime', values='TotalInvoice').fillna(0)
new_names = [(i, 'Daytime_Monetary_'+str(i)) for i in temp.columns.values]
temp.rename(columns = dict(new_names), inplace=True)
#temp = pd.get_dummies(invoicedb['InvoiceDate'].map(daysplit), prefix = 'Daytime')
invoicedb = invoicedb.merge(temp, how='inner', left_index=True, right_index=True)
invoicedb.head()

,InvoiceDate,TotalInvoice,TotalInvoiceInit,CustomerID,TotalSavings,AmountCancelled,QuantUnitPrice_01,QuantUnitPrice_02,QuantUnitPrice_03,QuantUnitPrice_04,...,QuantUnitSavings_02,QuantUnitSavings_03,QuantUnitSavings_04,QuantUnitSavings_05,Ordersep,Discount,Daytime,Daytime_Monetary_1,Daytime_Monetary_2,Daytime_Monetary_3
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,2010-12-01 08:26:00,139.12,139.12,17850.0,0.0,0.0,0.0,0.0,15.30,108.52,...,0.0,0.0,0.0,0.0,0,1.0,1,139.12,0.0,0.0
536366,2010-12-01 08:28:00,22.20,22.20,17850.0,0.0,0.0,0.0,0.0,22.20,0.00,...,0.0,0.0,0.0,0.0,0,1.0,1,22.20,0.0,0.0
536367,2010-12-01 08:34:00,278.73,278.73,13047.0,0.0,0.0,0.0,0.0,89.18,55.50,...,0.0,0.0,0.0,0.0,0,1.0,1,278.73,0.0,0.0
536368,2010-12-01 08:34:00,70.05,70.05,13047.0,0.0,0.0,0.0,0.0,0.00,25.50,...,0.0,0.0,0.0,0.0,0,1.0,1,70.05,0.0,0.0
536369,2010-12-01 08:35:00,17.85,17.85,13047.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0,1.0,1,17.85,0.0,0.0


In [48]:
#When the order has been placed during the week in pounds?
def daysplit(x):
    day = x.weekday()
    return 'Weekday_{}_{}'.format(day, list(calendar.day_name)[day])
    #return day

invoicedb['Weekday'] = invoicedb['InvoiceDate'].map(daysplit)
temp = invoicedb.pivot(columns='Weekday', values='TotalInvoice').fillna(0)
#temp.head()
invoicedb = invoicedb.merge(temp, how='inner', left_index=True, right_index=True)
invoicedb.head()



,InvoiceDate,TotalInvoice,TotalInvoiceInit,CustomerID,TotalSavings,AmountCancelled,QuantUnitPrice_01,QuantUnitPrice_02,QuantUnitPrice_03,QuantUnitPrice_04,...,Daytime_Monetary_1,Daytime_Monetary_2,Daytime_Monetary_3,Weekday,Weekday_0_Monday,Weekday_1_Tuesday,Weekday_2_Wednesday,Weekday_3_Thursday,Weekday_4_Friday,Weekday_6_Sunday
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,2010-12-01 08:26:00,139.12,139.12,17850.0,0.0,0.0,0.0,0.0,15.30,108.52,...,139.12,0.0,0.0,Weekday_2_Wednesday,0.0,0.0,139.12,0.0,0.0,0.0
536366,2010-12-01 08:28:00,22.20,22.20,17850.0,0.0,0.0,0.0,0.0,22.20,0.00,...,22.20,0.0,0.0,Weekday_2_Wednesday,0.0,0.0,22.20,0.0,0.0,0.0
536367,2010-12-01 08:34:00,278.73,278.73,13047.0,0.0,0.0,0.0,0.0,89.18,55.50,...,278.73,0.0,0.0,Weekday_2_Wednesday,0.0,0.0,278.73,0.0,0.0,0.0
536368,2010-12-01 08:34:00,70.05,70.05,13047.0,0.0,0.0,0.0,0.0,0.00,25.50,...,70.05,0.0,0.0,Weekday_2_Wednesday,0.0,0.0,70.05,0.0,0.0,0.0
536369,2010-12-01 08:35:00,17.85,17.85,13047.0,0.0,0.0,0.0,0.0,0.00,0.00,...,17.85,0.0,0.0,Weekday_2_Wednesday,0.0,0.0,17.85,0.0,0.0,0.0


In [49]:
#When the order has been placed during the month?
def daysplit(x):
    month = x.month
    return 'Month_{:02d}'.format(month)
    #return day

invoicedb['MonthOrder'] = invoicedb['InvoiceDate'].apply(daysplit)    
temp = invoicedb.pivot(columns='MonthOrder', values='TotalInvoice').fillna(0)   
#temp = pd.get_dummies(invoicedb['InvoiceDate'].map(daysplit))
#temp.head()
invoicedb = invoicedb.merge(temp, how='inner', left_index=True, right_index=True)
invoicedb.head()


,InvoiceDate,TotalInvoice,TotalInvoiceInit,CustomerID,TotalSavings,AmountCancelled,QuantUnitPrice_01,QuantUnitPrice_02,QuantUnitPrice_03,QuantUnitPrice_04,...,Month_03,Month_04,Month_05,Month_06,Month_07,Month_08,Month_09,Month_10,Month_11,Month_12
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,2010-12-01 08:26:00,139.12,139.12,17850.0,0.0,0.0,0.0,0.0,15.30,108.52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139.12
536366,2010-12-01 08:28:00,22.20,22.20,17850.0,0.0,0.0,0.0,0.0,22.20,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.20
536367,2010-12-01 08:34:00,278.73,278.73,13047.0,0.0,0.0,0.0,0.0,89.18,55.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,278.73
536368,2010-12-01 08:34:00,70.05,70.05,13047.0,0.0,0.0,0.0,0.0,0.00,25.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.05
536369,2010-12-01 08:35:00,17.85,17.85,13047.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.85


In [57]:
#Customerdb creation
date = invoicedb['InvoiceDate'].max()
def my_agg(x):
    aggcalcul = {
        'LastInvoice': x['InvoiceDate'].max(),
        'Recency': (date - x['InvoiceDate'].max()).days,
        'SpentMin': x['TotalInvoice'].min(),
        'SpentMax': x['TotalInvoice'].max(),
        'SpentMean': x['TotalInvoice'].mean(),
        'SpentSum': x['TotalInvoice'].sum(),
        'SpentStd': x['TotalInvoice'].std(),
        'OrderSepMean': x['Ordersep'].mean(),
        'OrderSepMax' : x['Ordersep'].max(),
        'OrderSepMin' : x['Ordersep'].min(),
        'OrderSepStd' : x['Ordersep'].std(),
        'Frequency' : x['InvoiceDate'].count(),
        'DiscountMean' : x['Discount'].mean(),
        'DiscountMax' : x['Discount'].max(),
        'DiscountMin' : x['Discount'].min(),
        'DiscountStd' : x['Discount'].std(),
        'SavingsSum' : x['TotalSavings'].sum(),
        'SavingsMean' : x['TotalSavings'].mean(),
        'SavingsMax' : x['TotalSavings'].max(),
        'SavingsMin' :x['TotalSavings'].min(), 
        'SavingsStd' : x['TotalSavings'].std(),
        'AmountCancelledSum' : x['AmountCancelled'].sum(),
        'AmountCancelledMean' : x['AmountCancelled'].mean(),
        'AmountCancelledMin' : x['AmountCancelled'].min(),
        'AmountCancelledMax' : x['AmountCancelled'].max(),
        'AmountCancelledStd' : x['AmountCancelled'].std(),
        }
    return pd.Series(aggcalcul, index=aggcalcul.keys())
customerdb = invoicedb.groupby('CustomerID').apply(my_agg).fillna(0)

customerdb.head()

,LastInvoice,Recency,SpentMin,SpentMax,SpentMean,SpentSum,SpentStd,OrderSepMean,OrderSepMax,OrderSepMin,...,SavingsSum,SavingsMean,SavingsMax,SavingsMin,SavingsStd,AmountCancelledSum,AmountCancelledMean,AmountCancelledMin,AmountCancelledMax,AmountCancelledStd
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,2011-01-18 10:17:00,325,-77183.60,77183.60,0.000000,0.00,109154.093913,0.00000,0,0,...,0.00,0.000000,0.00,0.00,0.000000,-77183.60,-38591.80000,-77183.60,0.0,54577.046956
12747.0,2011-12-07 14:34:00,1,301.70,675.38,381.455455,4196.01,109.620426,33.00000,65,0,...,83.12,7.556364,40.20,-32.40,23.861230,0.00,0.00000,0.00,0.0,0.000000
12748.0,2011-12-09 12:20:00,0,-1301.00,1948.35,137.419309,29819.99,255.191237,1.37788,30,0,...,-11985.18,-55.231244,4452.50,-7780.50,688.188713,-1830.79,-8.43682,-1301.00,0.0,89.841910
12749.0,2011-12-06 09:56:00,3,-125.78,1866.43,477.275000,3818.20,684.279697,25.75000,91,0,...,115.47,14.433750,41.64,-3.29,18.942951,-222.68,-27.83500,-125.78,0.0,47.730375
12820.0,2011-12-06 15:12:00,2,170.46,343.76,235.585000,942.34,75.049485,80.50000,251,0,...,15.28,3.820000,4.80,1.68,1.439074,0.00,0.00000,0.00,0.0,0.000000


In [58]:
#detail orders for customerdb QUANT UNIT PRICE
detail = [x for x in invoicedb.columns.values if x.startswith('QuantUnitPrice_')]
detail.append('CustomerID')
temp = invoicedb[detail].groupby('CustomerID').agg([np.sum, np.min, np.max, np.std]).fillna(0)
newnames = ["_".join(x) for x in temp.columns.ravel()]
temp.columns = newnames
temp.head()
customerdb = customerdb.merge(temp, how='inner', left_index=True, right_index=True)
customerdb.head()

,LastInvoice,Recency,SpentMin,SpentMax,SpentMean,SpentSum,SpentStd,OrderSepMean,OrderSepMax,OrderSepMin,...,QuantUnitPrice_03_amax,QuantUnitPrice_03_std,QuantUnitPrice_04_sum,QuantUnitPrice_04_amin,QuantUnitPrice_04_amax,QuantUnitPrice_04_std,QuantUnitPrice_05_sum,QuantUnitPrice_05_amin,QuantUnitPrice_05_amax,QuantUnitPrice_05_std
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,2011-01-18 10:17:00,325,-77183.60,77183.60,0.000000,0.00,109154.093913,0.00000,0,0,...,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000
12747.0,2011-12-07 14:34:00,1,301.70,675.38,381.455455,4196.01,109.620426,33.00000,65,0,...,367.28,84.033280,77.00,0.00,17.00,8.062258,2273.75,55.70,308.10,70.522867
12748.0,2011-12-09 12:20:00,0,-1301.00,1948.35,137.419309,29819.99,255.191237,1.37788,30,0,...,544.85,63.602389,6175.49,-485.00,550.00,75.539243,7149.46,-816.00,816.00,108.178653
12749.0,2011-12-06 09:56:00,3,-125.78,1866.43,477.275000,3818.20,684.279697,25.75000,91,0,...,194.00,85.100797,970.14,-30.22,372.06,155.131776,1704.30,-70.35,832.25,307.592806
12820.0,2011-12-06 15:12:00,2,170.46,343.76,235.585000,942.34,75.049485,80.50000,251,0,...,111.84,28.767609,180.34,17.70,73.04,22.946334,69.30,0.00,24.75,11.783357


In [59]:
#detail orders for customerdb QUANT SAVINGS
detail = [x for x in invoicedb.columns.values if x.startswith('QuantUnitSavings_')]
detail.append('CustomerID')
temp = invoicedb[detail].groupby('CustomerID').agg([np.sum, np.min, np.max, np.std]).fillna(0)
newnames = ["_".join(x) for x in temp.columns.ravel()]
temp.columns = newnames
temp.head()
customerdb = customerdb.merge(temp, how='inner', left_index=True, right_index=True)
customerdb.head()

,LastInvoice,Recency,SpentMin,SpentMax,SpentMean,SpentSum,SpentStd,OrderSepMean,OrderSepMax,OrderSepMin,...,QuantUnitSavings_03_amax,QuantUnitSavings_03_std,QuantUnitSavings_04_sum,QuantUnitSavings_04_amin,QuantUnitSavings_04_amax,QuantUnitSavings_04_std,QuantUnitSavings_05_sum,QuantUnitSavings_05_amin,QuantUnitSavings_05_amax,QuantUnitSavings_05_std
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,2011-01-18 10:17:00,325,-77183.60,77183.60,0.000000,0.00,109154.093913,0.00000,0,0,...,0.00,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000
12747.0,2011-12-07 14:34:00,1,301.70,675.38,381.455455,4196.01,109.620426,33.00000,65,0,...,27.90,9.873481,0.00,0.0,0.0,0.000000,-22.70,-32.4,12.3,18.049503
12748.0,2011-12-09 12:20:00,0,-1301.00,1948.35,137.419309,29819.99,255.191237,1.37788,30,0,...,21.01,59.050292,-5434.19,-4125.0,2023.1,344.136293,-3893.62,-3588.0,2429.4,337.740497
12749.0,2011-12-06 09:56:00,3,-125.78,1866.43,477.275000,3818.20,684.279697,25.75000,91,0,...,7.80,2.766771,26.90,-0.8,10.5,4.292830,54.80,-2.0,30.8,11.668150
12820.0,2011-12-06 15:12:00,2,170.46,343.76,235.585000,942.34,75.049485,80.50000,251,0,...,4.80,1.959592,2.00,0.0,2.0,1.000000,0.00,0.0,0.0,0.000000


In [60]:
#Time of the day aggregation
detail = [x for x in invoicedb.columns.values if x.startswith('Daytime_Monetary_')]
detail.append('CustomerID')
temp = invoicedb[detail].groupby('CustomerID').agg([np.sum, np.mean, np.std]).fillna(0)
newnames = ["_".join(x) for x in temp.columns.ravel()]
#temp.columns = temp.columns.droplevel(0)
temp.columns = newnames
temp.head()
customerdb = customerdb.merge(temp, how='inner', left_index=True, right_index=True)
customerdb.head()

,LastInvoice,Recency,SpentMin,SpentMax,SpentMean,SpentSum,SpentStd,OrderSepMean,OrderSepMax,OrderSepMin,...,QuantUnitSavings_05_std,Daytime_Monetary_1_sum,Daytime_Monetary_1_mean,Daytime_Monetary_1_std,Daytime_Monetary_2_sum,Daytime_Monetary_2_mean,Daytime_Monetary_2_std,Daytime_Monetary_3_sum,Daytime_Monetary_3_mean,Daytime_Monetary_3_std
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,2011-01-18 10:17:00,325,-77183.60,77183.60,0.000000,0.00,109154.093913,0.00000,0,0,...,0.000000,0.00,0.000000,109154.093913,0.00,0.000000,0.000000,0.00,0.000000,0.000000
12747.0,2011-12-07 14:34:00,1,301.70,675.38,381.455455,4196.01,109.620426,33.00000,65,0,...,18.049503,2029.44,184.494545,233.036066,2166.57,196.960909,194.048703,0.00,0.000000,0.000000
12748.0,2011-12-09 12:20:00,0,-1301.00,1948.35,137.419309,29819.99,255.191237,1.37788,30,0,...,337.740497,4898.29,22.572765,72.209675,24840.96,114.474470,255.310826,80.74,0.372074,4.370894
12749.0,2011-12-06 09:56:00,3,-125.78,1866.43,477.275000,3818.20,684.279697,25.75000,91,0,...,11.668150,637.28,79.660000,279.621031,3200.82,400.102500,678.324519,-19.90,-2.487500,7.035712
12820.0,2011-12-06 15:12:00,2,170.46,343.76,235.585000,942.34,75.049485,80.50000,251,0,...,0.000000,217.77,54.442500,108.885000,724.57,181.142500,141.685391,0.00,0.000000,0.000000


In [61]:
#Time of the month aggregation
detail = [x for x in invoicedb.columns.values if x.startswith('Month_')]
detail.append('CustomerID')
temp = invoicedb[detail].groupby('CustomerID').agg([np.sum, np.mean, np.std]).fillna(0)
newnames = ["_".join(x) for x in temp.columns.ravel()]
temp.columns = newnames
temp.head()
customerdb = customerdb.merge(temp, how='inner', left_index=True, right_index=True)
customerdb.head()

,LastInvoice,Recency,SpentMin,SpentMax,SpentMean,SpentSum,SpentStd,OrderSepMean,OrderSepMax,OrderSepMin,...,Month_09_std,Month_10_sum,Month_10_mean,Month_10_std,Month_11_sum,Month_11_mean,Month_11_std,Month_12_sum,Month_12_mean,Month_12_std
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,2011-01-18 10:17:00,325,-77183.60,77183.60,0.000000,0.00,109154.093913,0.00000,0,0,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000
12747.0,2011-12-07 14:34:00,1,301.70,675.38,381.455455,4196.01,109.620426,33.00000,65,0,...,0.000000,675.38,61.398182,203.634732,312.73,28.430000,94.291643,1144.77,104.070000,179.614463
12748.0,2011-12-09 12:20:00,0,-1301.00,1948.35,137.419309,29819.99,255.191237,1.37788,30,0,...,184.042233,2149.30,9.904608,69.792212,8257.83,38.054516,158.866489,5090.47,23.458387,76.161941
12749.0,2011-12-06 09:56:00,3,-125.78,1866.43,477.275000,3818.20,684.279697,25.75000,91,0,...,0.000000,0.00,0.000000,0.000000,522.59,65.323750,184.763466,763.06,95.382500,269.782450
12820.0,2011-12-06 15:12:00,2,170.46,343.76,235.585000,942.34,75.049485,80.50000,251,0,...,108.885000,343.76,85.940000,171.880000,0.00,0.000000,0.000000,210.35,52.587500,105.175000


In [62]:
#Time of the week aggregation
detail = [x for x in invoicedb.columns.values if x.startswith('Weekday_')]
detail.append('CustomerID')
temp = invoicedb[detail].groupby('CustomerID').agg([np.sum, np.mean, np.std]).fillna(0)
newnames = ["_".join(x) for x in temp.columns.ravel()]
temp.columns = newnames
#temp.head()
customerdb = customerdb.merge(temp, how='inner', left_index=True, right_index=True)
customerdb.head()

,LastInvoice,Recency,SpentMin,SpentMax,SpentMean,SpentSum,SpentStd,OrderSepMean,OrderSepMax,OrderSepMin,...,Weekday_2_Wednesday_std,Weekday_3_Thursday_sum,Weekday_3_Thursday_mean,Weekday_3_Thursday_std,Weekday_4_Friday_sum,Weekday_4_Friday_mean,Weekday_4_Friday_std,Weekday_6_Sunday_sum,Weekday_6_Sunday_mean,Weekday_6_Sunday_std
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,2011-01-18 10:17:00,325,-77183.60,77183.60,0.000000,0.00,109154.093913,0.00000,0,0,...,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000
12747.0,2011-12-07 14:34:00,1,301.70,675.38,381.455455,4196.01,109.620426,33.00000,65,0,...,157.046512,1058.73,96.248182,168.506979,0.00,0.000000,0.000000,358.56,32.596364,108.109908
12748.0,2011-12-09 12:20:00,0,-1301.00,1948.35,137.419309,29819.99,255.191237,1.37788,30,0,...,167.538969,4140.04,19.078525,66.743641,7530.98,34.704977,118.293336,4837.17,22.291106,146.068739
12749.0,2011-12-06 09:56:00,3,-125.78,1866.43,477.275000,3818.20,684.279697,25.75000,91,0,...,0.000000,376.91,47.113750,196.962391,0.00,0.000000,0.000000,0.00,0.000000,0.000000
12820.0,2011-12-06 15:12:00,2,170.46,343.76,235.585000,942.34,75.049485,80.50000,251,0,...,171.880000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000


In [66]:
#rfm score calculation
scorerange = 4
divider = 100/scorerange
customerdb['F'] = customerdb['Frequency'].apply(lambda x: scorerange + 1 - math.ceil(stats.percentileofscore(customerdb['Frequency'],x, kind = 'mean')/divider))
customerdb['R'] = customerdb['Recency'].apply(lambda x: math.ceil(stats.percentileofscore(customerdb['Recency'],x, kind = 'mean')/divider))
customerdb['M'] = customerdb['SpentSum'].apply(lambda x: scorerange + 1 - math.ceil(stats.percentileofscore(customerdb['SpentSum'],x, kind = 'mean')/divider))
customerdb['D'] = customerdb['SavingsSum'].apply(lambda x: scorerange + 1 - math.ceil(stats.percentileofscore(customerdb['SavingsSum'],x, kind = 'mean')/divider))

scorerange = 3
divider = 100/scorerange
customerdb['C'] = customerdb['AmountCancelledSum'].apply(lambda x: scorerange + 1 - math.ceil(stats.percentileofscore(customerdb['AmountCancelledSum'],x, kind = 'mean')/divider))
customerdb['RFMDC'] = customerdb['R'] *10000 +  customerdb['F'] *1000 +  customerdb['M'] *100 + customerdb['D'] *10 + customerdb['C'] *1
customerdb[['R', 'F', 'M', 'D', 'C', 'RFMDC']].head()

,R,F,M,D,C,RFMDC
CustomerID,,,,,,
12346.0,4,3,4,2,3,43423
12747.0,1,1,1,1,1,11111
12748.0,1,1,1,4,3,11143
12749.0,1,1,1,1,3,11113
12820.0,1,2,2,1,1,12211


In [67]:
customerdb[['R', 'F', 'M', 'D', 'C', 'RFMDC']].describe()
#customerdb.columns.values

,R,F,M,D,C,RFMDC
count,3942.000000,3942.000000,3942.000000,3942.000000,3942.000000,3942.000000
mean,2.509132,2.547438,2.500254,2.500254,1.683156,27915.473110
std,1.112907,1.160177,1.118176,1.118176,0.940134,11819.771037
min,1.000000,1.000000,1.000000,1.000000,1.000000,11111.000000
25%,2.000000,2.000000,2.000000,2.000000,1.000000,21113.000000
50%,3.000000,2.000000,2.500000,2.500000,1.000000,31113.000000
75%,3.000000,4.000000,3.750000,3.750000,3.000000,34441.000000
max,4.000000,4.000000,4.000000,4.000000,3.000000,44441.000000


In [68]:
def customerdbcreation(invoiceset, light = True, pitch = 1, monthcovered = 4):
    dataset = pd.DataFrame()
    dates = np.sort(invoiceset['InvoiceDate'].map(lambda x : x.date()).unique())
    #We drop the first days
    dates = np.delete(dates, [0,1])
    counter = 0
    for orderdate in dates[::pitch]:
        counter += 1
        df = invoiceset[invoiceset['InvoiceDate'] <= orderdate]
        df = df[df['InvoiceDate'] > orderdate - timedelta(days=30 * monthcovered)]
        
        date = df['InvoiceDate'].max()
        def my_agg(x):
            aggcalcul = {
                    'LastInvoice': x['InvoiceDate'].max(),
                    'Recency': (date - x['InvoiceDate'].max()).days,
                    'SpentMin': x['TotalInvoice'].min(),
                    'SpentMax': x['TotalInvoice'].max(),
                    'SpentMean': x['TotalInvoice'].mean(),
                    'SpentSum': x['TotalInvoice'].sum(),
                    'SpentStd': x['TotalInvoice'].std(),
                    'OrderSepMean': x['Ordersep'].mean(),
                    'OrderSepMax' : x['Ordersep'].max(),
                    'OrderSepMin' : x['Ordersep'].min(),
                    'OrderSepStd' : x['Ordersep'].std(),
                    'Frequency' : x['InvoiceDate'].count(),
                    'DiscountMean' : x['Discount'].mean(),
                    'DiscountMax' : x['Discount'].max(),
                    'DiscountMin' : x['Discount'].min(),
                    'DiscountStd' : x['Discount'].std(),
                    'SavingsSum' : x['TotalSavings'].sum(),
                    'SavingsMean' : x['TotalSavings'].mean(),
                    'SavingsMax' : x['TotalSavings'].max(),
                    'SavingsMin' :x['TotalSavings'].min(), 
                    'SavingsStd' : x['TotalSavings'].std(),
                    'AmountCancelledSum' : x['AmountCancelled'].sum(),
                    'AmountCancelledMean' : x['AmountCancelled'].mean(),
                    'AmountCancelledMin' : x['AmountCancelled'].min(),
                    'AmountCancelledMax' : x['AmountCancelled'].max(),
                    'AmountCancelledStd' : x['AmountCancelled'].std(),
                    }
            return pd.Series(aggcalcul, index=aggcalcul.keys())
        customer = df.groupby('CustomerID').apply(my_agg).fillna(0)


        if not light:
            #detail orders for customerdb QUANT UNIT PRICE
            detail = [x for x in df.columns.values if x.startswith('QuantUnitPrice_')]
            detail.append('CustomerID')
            temp = df[detail].groupby('CustomerID').agg([np.sum, np.min, np.max, np.std]).fillna(0)
            newnames = ["_".join(x) for x in temp.columns.ravel()]
            temp.columns = newnames
            customer = customer.merge(temp, how='inner', left_index=True, right_index=True)

            #detail orders for customerdb QUANT SAVINGS
            detail = [x for x in df.columns.values if x.startswith('QuantUnitSavings_')]
            detail.append('CustomerID')
            temp = df[detail].groupby('CustomerID').agg([np.sum, np.min, np.max, np.std]).fillna(0)
            newnames = ["_".join(x) for x in temp.columns.ravel()]
            temp.columns = newnames
            customer = customer.merge(temp, how='inner', left_index=True, right_index=True)

            #Time of the day aggregation
            detail = [x for x in df.columns.values if x.startswith('Daytime_Monetary_')]
            detail.append('CustomerID')
            temp = df[detail].groupby('CustomerID').agg([np.sum, np.mean, np.std]).fillna(0)
            newnames = ["_".join(x) for x in temp.columns.ravel()]
            temp.columns = newnames
            customer = customer.merge(temp, how='inner', left_index=True, right_index=True)

            #Time of the month aggregation
            detail = [x for x in df.columns.values if x.startswith('Month_')]
            detail.append('CustomerID')
            temp = df[detail].groupby('CustomerID').agg([np.sum, np.mean, np.std]).fillna(0)
            newnames = ["_".join(x) for x in temp.columns.ravel()]
            temp.columns = newnames
            customer = customer.merge(temp, how='inner', left_index=True, right_index=True)
            customer.head()
            
            #Time of the week aggregation
            detail = [x for x in df.columns.values if x.startswith('Weekday_')]
            detail.append('CustomerID')
            temp = df[detail].groupby('CustomerID').agg([np.sum, np.mean, np.std]).fillna(0)
            newnames = ["_".join(x) for x in temp.columns.ravel()]
            temp.columns = newnames
            customer = customer.merge(temp, how='inner', left_index=True, right_index=True)

        
        #rfm score calculation
        scorerange = 4
        divider = 100/scorerange
        customer['F'] = customer['Frequency'].apply(lambda x: scorerange + 1 - math.ceil(stats.percentileofscore(customer['Frequency'],x, kind = 'mean')/divider))
        customer['R'] = customer['Recency'].apply(lambda x: math.ceil(stats.percentileofscore(customer['Recency'],x, kind = 'mean')/divider))
        customer['M'] = customer['SpentSum'].apply(lambda x: scorerange + 1 - math.ceil(stats.percentileofscore(customer['SpentSum'],x, kind = 'mean')/divider))
        customer['D'] = customer['SavingsSum'].apply(lambda x: scorerange + 1 - math.ceil(stats.percentileofscore(customer['SavingsSum'],x, kind = 'mean')/divider))
        
        scorerange = 3
        divider = 100/scorerange
        customer['C'] = customer['AmountCancelledSum'].apply(lambda x: scorerange + 1 - math.ceil(stats.percentileofscore(customer['AmountCancelledSum'],x, kind = 'mean')/divider))
        customer['RFMDC'] = customer['R'] *10000 +  customer['F'] *1000 +  customer['M'] *100 + customer['D'] *10 + customer['C'] *1
        
        #Timestamp on current day set
        customer['Timestamp'] = customer['LastInvoice'].apply(lambda x: orderdate)
        
        #Data assembly
        print('{}-{} - {}'.format(counter, len(dates[::pitch]), orderdate))
        dataset = pd.concat([dataset, customer.reset_index()], axis = 0)
        print('dataset size : ', dataset.shape)
        print('Timestamp -->', datetime.now())
        
    return dataset
    
dataset = customerdbcreation(invoicedb.head(1000), light = False, pitch = 2, monthcovered=4)
#dataset.to_csv('Z1 - completepicturecustomer1yearP2.csv')
dataset.head()

1-5 - 2010-12-03
dataset size :  (194, 137)
Timestamp --> 2018-01-06 14:28:01.997283
2-5 - 2010-12-06
dataset size :  (490, 137)
Timestamp --> 2018-01-06 14:28:02.996607
3-5 - 2010-12-08
dataset size :  (908, 137)
Timestamp --> 2018-01-06 14:28:04.294526
4-5 - 2010-12-10
dataset size :  (1481, 137)
Timestamp --> 2018-01-06 14:28:06.124174
5-5 - 2010-12-13
dataset size :  (2120, 137)
Timestamp --> 2018-01-06 14:28:08.182139


,CustomerID,LastInvoice,Recency,SpentMin,SpentMax,SpentMean,SpentSum,SpentStd,OrderSepMean,OrderSepMax,...,Weekday_6_Sunday_sum,Weekday_6_Sunday_mean,Weekday_6_Sunday_std,F,R,M,D,C,RFMDC,Timestamp
0,12748.0,2010-12-02 11:46:00,0,4.25,4.95,4.60,9.20,0.494975,0.0,0,...,0.0,0.0,0.0,1,2,4,3,2,21432,2010-12-03
1,12838.0,2010-12-01 11:57:00,1,390.79,390.79,390.79,390.79,0.000000,0.0,0,...,0.0,0.0,0.0,3,4,1,1,2,43112,2010-12-03
2,12855.0,2010-12-02 09:37:00,0,38.10,38.10,38.10,38.10,0.000000,0.0,0,...,0.0,0.0,0.0,3,2,4,2,2,23422,2010-12-03
3,12868.0,2010-12-01 12:50:00,1,203.30,203.30,203.30,203.30,0.000000,0.0,0,...,0.0,0.0,0.0,3,4,3,3,2,43332,2010-12-03
4,12915.0,2010-12-02 12:33:00,0,199.65,199.65,199.65,199.65,0.000000,0.0,0,...,0.0,0.0,0.0,3,2,3,2,2,23322,2010-12-03


In [69]:
#Training set creation
dataset = customerdbcreation(invoicedb, light = False, pitch = 3, monthcovered=4)
dataset.to_csv('Z1 - completepicturecustomer1yearP3MC4.csv')
dataset.head()

1-101 - 2010-12-03
dataset size :  (194, 137)
Timestamp --> 2018-01-06 14:28:37.760678
2-101 - 2010-12-07
dataset size :  (562, 137)
Timestamp --> 2018-01-06 14:28:39.042775
3-101 - 2010-12-10
dataset size :  (1135, 137)
Timestamp --> 2018-01-06 14:28:40.876312
4-101 - 2010-12-14
dataset size :  (1806, 137)
Timestamp --> 2018-01-06 14:28:43.049885
5-101 - 2010-12-17
dataset size :  (2620, 137)
Timestamp --> 2018-01-06 14:28:45.592289
6-101 - 2010-12-21
dataset size :  (3471, 137)
Timestamp --> 2018-01-06 14:28:48.320649
7-101 - 2011-01-04
dataset size :  (4341, 137)
Timestamp --> 2018-01-06 14:28:51.057837
8-101 - 2011-01-07
dataset size :  (5256, 137)
Timestamp --> 2018-01-06 14:28:53.873343
9-101 - 2011-01-11
dataset size :  (6222, 137)
Timestamp --> 2018-01-06 14:28:56.849801
10-101 - 2011-01-14
dataset size :  (7239, 137)
Timestamp --> 2018-01-06 14:28:59.994741
11-101 - 2011-01-18
dataset size :  (8295, 137)
Timestamp --> 2018-01-06 14:29:03.211493
12-101 - 2011-01-21
dataset size

92-101 - 2011-11-06
dataset size :  (167447, 137)
Timestamp --> 2018-01-06 14:37:41.216614
93-101 - 2011-11-09
dataset size :  (170001, 137)
Timestamp --> 2018-01-06 14:37:49.567429
94-101 - 2011-11-13
dataset size :  (172588, 137)
Timestamp --> 2018-01-06 14:37:57.941312
95-101 - 2011-11-16
dataset size :  (175223, 137)
Timestamp --> 2018-01-06 14:38:06.642262
96-101 - 2011-11-20
dataset size :  (177893, 137)
Timestamp --> 2018-01-06 14:38:15.396913
97-101 - 2011-11-23
dataset size :  (180617, 137)
Timestamp --> 2018-01-06 14:38:24.183192
98-101 - 2011-11-27
dataset size :  (183362, 137)
Timestamp --> 2018-01-06 14:38:33.127754
99-101 - 2011-11-30
dataset size :  (186149, 137)
Timestamp --> 2018-01-06 14:38:42.382583
100-101 - 2011-12-04
dataset size :  (188941, 137)
Timestamp --> 2018-01-06 14:38:51.508684
101-101 - 2011-12-07
dataset size :  (191764, 137)
Timestamp --> 2018-01-06 14:39:01.042497


,CustomerID,LastInvoice,Recency,SpentMin,SpentMax,SpentMean,SpentSum,SpentStd,OrderSepMean,OrderSepMax,...,Weekday_6_Sunday_sum,Weekday_6_Sunday_mean,Weekday_6_Sunday_std,F,R,M,D,C,RFMDC,Timestamp
0,12748.0,2010-12-02 11:46:00,0,4.25,4.95,4.60,9.20,0.494975,0.0,0,...,0.0,0.0,0.0,1,2,4,3,2,21432,2010-12-03
1,12838.0,2010-12-01 11:57:00,1,390.79,390.79,390.79,390.79,0.000000,0.0,0,...,0.0,0.0,0.0,3,4,1,1,2,43112,2010-12-03
2,12855.0,2010-12-02 09:37:00,0,38.10,38.10,38.10,38.10,0.000000,0.0,0,...,0.0,0.0,0.0,3,2,4,2,2,23422,2010-12-03
3,12868.0,2010-12-01 12:50:00,1,203.30,203.30,203.30,203.30,0.000000,0.0,0,...,0.0,0.0,0.0,3,4,3,3,2,43332,2010-12-03
4,12915.0,2010-12-02 12:33:00,0,199.65,199.65,199.65,199.65,0.000000,0.0,0,...,0.0,0.0,0.0,3,2,3,2,2,23322,2010-12-03


In [70]:
#customer behaving calculation analysis
dataset = customerdbcreation(invoicedb, light = True, pitch = 1, monthcovered=20)
dataset.to_csv('Z1 - completepicturecustomer1yearP1MC20.csv')
dataset.head()

1-303 - 2010-12-03
dataset size :  (194, 34)
Timestamp --> 2018-01-06 14:39:31.562677
2-303 - 2010-12-05
dataset size :  (426, 34)
Timestamp --> 2018-01-06 14:39:32.315236
3-303 - 2010-12-06
dataset size :  (722, 34)
Timestamp --> 2018-01-06 14:39:33.286390
4-303 - 2010-12-07
dataset size :  (1090, 34)
Timestamp --> 2018-01-06 14:39:34.505151
5-303 - 2010-12-08
dataset size :  (1508, 34)
Timestamp --> 2018-01-06 14:39:35.814972
6-303 - 2010-12-09
dataset size :  (2010, 34)
Timestamp --> 2018-01-06 14:39:37.454069
7-303 - 2010-12-10
dataset size :  (2583, 34)
Timestamp --> 2018-01-06 14:39:39.291717
8-303 - 2010-12-12
dataset size :  (3192, 34)
Timestamp --> 2018-01-06 14:39:41.266414
9-303 - 2010-12-13
dataset size :  (3831, 34)
Timestamp --> 2018-01-06 14:39:43.346521
10-303 - 2010-12-14
dataset size :  (4502, 34)
Timestamp --> 2018-01-06 14:39:45.522569
11-303 - 2010-12-15
dataset size :  (5213, 34)
Timestamp --> 2018-01-06 14:39:47.832892
12-303 - 2010-12-16
dataset size :  (5970, 3

94-303 - 2011-04-01
dataset size :  (119258, 34)
Timestamp --> 2018-01-06 14:45:49.170324
95-303 - 2011-04-03
dataset size :  (121250, 34)
Timestamp --> 2018-01-06 14:45:55.581456
96-303 - 2011-04-04
dataset size :  (123245, 34)
Timestamp --> 2018-01-06 14:46:01.999225
97-303 - 2011-04-05
dataset size :  (125255, 34)
Timestamp --> 2018-01-06 14:46:08.382417
98-303 - 2011-04-06
dataset size :  (127278, 34)
Timestamp --> 2018-01-06 14:46:14.792582
99-303 - 2011-04-07
dataset size :  (129315, 34)
Timestamp --> 2018-01-06 14:46:21.423153
100-303 - 2011-04-08
dataset size :  (131370, 34)
Timestamp --> 2018-01-06 14:46:27.817676
101-303 - 2011-04-10
dataset size :  (133444, 34)
Timestamp --> 2018-01-06 14:46:34.282952
102-303 - 2011-04-11
dataset size :  (135527, 34)
Timestamp --> 2018-01-06 14:46:40.647641
103-303 - 2011-04-12
dataset size :  (137625, 34)
Timestamp --> 2018-01-06 14:46:47.275516
104-303 - 2011-04-13
dataset size :  (139738, 34)
Timestamp --> 2018-01-06 14:46:54.028247
105-3

185-303 - 2011-07-24
dataset size :  (344430, 34)
Timestamp --> 2018-01-06 14:57:42.400849
186-303 - 2011-07-25
dataset size :  (347268, 34)
Timestamp --> 2018-01-06 14:57:51.601242
187-303 - 2011-07-26
dataset size :  (350113, 34)
Timestamp --> 2018-01-06 14:58:00.447586
188-303 - 2011-07-27
dataset size :  (352961, 34)
Timestamp --> 2018-01-06 14:58:09.329022
189-303 - 2011-07-28
dataset size :  (355814, 34)
Timestamp --> 2018-01-06 14:58:18.078268
190-303 - 2011-07-29
dataset size :  (358675, 34)
Timestamp --> 2018-01-06 14:58:26.928928
191-303 - 2011-07-31
dataset size :  (361549, 34)
Timestamp --> 2018-01-06 14:58:35.847751
192-303 - 2011-08-01
dataset size :  (364427, 34)
Timestamp --> 2018-01-06 14:58:44.923833
193-303 - 2011-08-02
dataset size :  (367312, 34)
Timestamp --> 2018-01-06 14:58:53.998109
194-303 - 2011-08-03
dataset size :  (370206, 34)
Timestamp --> 2018-01-06 14:59:03.400612
195-303 - 2011-08-04
dataset size :  (373108, 34)
Timestamp --> 2018-01-06 14:59:12.195111

dataset size :  (631144, 34)
Timestamp --> 2018-01-06 15:12:53.105122
276-303 - 2011-11-08
dataset size :  (634834, 34)
Timestamp --> 2018-01-06 15:13:06.015439
277-303 - 2011-11-09
dataset size :  (638536, 34)
Timestamp --> 2018-01-06 15:13:18.954190
278-303 - 2011-11-10
dataset size :  (642255, 34)
Timestamp --> 2018-01-06 15:13:31.494990
279-303 - 2011-11-11
dataset size :  (645985, 34)
Timestamp --> 2018-01-06 15:13:43.908119
280-303 - 2011-11-13
dataset size :  (649726, 34)
Timestamp --> 2018-01-06 15:13:56.053337
281-303 - 2011-11-14
dataset size :  (653475, 34)
Timestamp --> 2018-01-06 15:14:08.496069
282-303 - 2011-11-15
dataset size :  (657240, 34)
Timestamp --> 2018-01-06 15:14:21.066762
283-303 - 2011-11-16
dataset size :  (661020, 34)
Timestamp --> 2018-01-06 15:14:33.758845
284-303 - 2011-11-17
dataset size :  (664811, 34)
Timestamp --> 2018-01-06 15:14:46.140000
285-303 - 2011-11-18
dataset size :  (668611, 34)
Timestamp --> 2018-01-06 15:14:58.843252
286-303 - 2011-11-20

,CustomerID,LastInvoice,Recency,SpentMin,SpentMax,SpentMean,SpentSum,SpentStd,OrderSepMean,OrderSepMax,...,AmountCancelledMin,AmountCancelledMax,AmountCancelledStd,F,R,M,D,C,RFMDC,Timestamp
0,12748.0,2010-12-02 11:46:00,0,4.25,4.95,4.60,9.20,0.494975,0.0,0,...,0.0,0.0,0.0,1,2,4,3,2,21432,2010-12-03
1,12838.0,2010-12-01 11:57:00,1,390.79,390.79,390.79,390.79,0.000000,0.0,0,...,0.0,0.0,0.0,3,4,1,1,2,43112,2010-12-03
2,12855.0,2010-12-02 09:37:00,0,38.10,38.10,38.10,38.10,0.000000,0.0,0,...,0.0,0.0,0.0,3,2,4,2,2,23422,2010-12-03
3,12868.0,2010-12-01 12:50:00,1,203.30,203.30,203.30,203.30,0.000000,0.0,0,...,0.0,0.0,0.0,3,4,3,3,2,43332,2010-12-03
4,12915.0,2010-12-02 12:33:00,0,199.65,199.65,199.65,199.65,0.000000,0.0,0,...,0.0,0.0,0.0,3,2,3,2,2,23322,2010-12-03


In [ ]:
customerdb.shape